# Mobile Robotics Project

#### Group 10:  
Amine Tourki  
Antoine Jérémie Delaloye  
Ruben Jungius  
Marco Angelo Pontarolo  

## Introduction

The goal of this project is to navigate a Thymio robot through an environment with obstacles in order to reach a goal position. This project includes four major modules: Global and Local Navigation, Vision and Filtering. 

## Setup
The setup for our project is very simple. It consists of an open arena (no marked boundaries), with black obstacles made of paper, a colored ground (red or green) and a simple flat cirle as an objective. Discussion about the choices can be found in the vision module.  
(*insert a good setup image here*)

## Vision
The vision module in this project is responsible for detecting the obstacles and the goal, the state of the Thymio (position and orientation) as well as constructing a map for the Global Planning module. Most of the vision module is done using the OpenCV library. Below are the choices made for the Arena:

| **Choice**          | **Reason**                                                                                                                          |
|---------------------|-------------------------------------------------------------------------------------------------------------------------------------|
| Open arena          | The arena was kept open in order to avoid any unnecesary errors due to the hough transform or any other lign detection algorithm    |
| Black obstacles     | The obstacles were chosen to be black for the ease of detection using thresholding                                                  |
| Round red objective | A circular form was found to be the easiest to detect. The color was chosen red to create an apparent gradient from the map's floor |
| Green floor         | The color of the floor was chosen in order to have a great contrast from the obstacles, making their detection much easier          |

### General vision class
The vision module was made into a class to make the extraction of information much easier and add structure to the whole module. An instance of the class is created and can be used to access information about the vision at any time. The main functions provided by the Vision are as follow:

| **Function**           | **Role**                                                                  |
|------------------------|---------------------------------------------------------------------------|
| vision.openWindow()    | Opens an independent window with the vision of the camera                 |
| vision.getRobotPos()   | Returns the current position of the robot in the map (in term of cells)   |
| vision.getRobotAngle() | Returns the orientation of the robot (clockwise from the horizontal axis) |
| vision.getMap()        | Returns a binary map of the arena (1 for obstacles, 0 is free)            |
| vision.getGoal()       | Return the position of the goal in the map                                |

### Thymio detection
The detection of the thymio is done using thresholding. A mask is generated from the threshoding which is then further processed to remove any noise. The contour of the thymio can then be and used to extract both the position of the center using the moment of the contour and the orientation by fitting a rectangle on the Thymio and getting the orientation of the fitted rectangle. Follow are the main functions from CV2 used for each part:
- Thresholding: `cv2.threshold`
- Mask processing: `cv2.morphologyEx`
- Finding the contour : `cv2.findContours`
- Calculating the moment of the contour: `cv2.moments`
- Fitting a rectangle: `cv2.minAreaRect`

Here is an example of the Thymio detection steps:


<center><img src="images/vision_thymio_in_arena.PNG"></center>
<center>Original image from the camera</center>

<center><img src="images/vision_thymio_threshold_noisy.png"></center>
<center>Thresholding the image to botain the Thymio</center>

<center><img src="images\vision_thymio_threshold_processed.png"></center>
<center>Denoising the mask</center>

<center><img src="images\vision_thymio_contour.png"></center>
<center>Extracting the contour of the Thymio from the mask</center>

<center><img src="images\vision_thymio_contour_center.png"></center>
<center>Calculate the center and orientation of the Thymio using the contour</center>

[//]: <> (<center><img src=images/vision_thymio_in_arena.png></center>)
[//]: <> (<center>Original image from the camera</center>)

The function responsible for updating the robot position and orientation is implemented as follows: 
```python
def updateRobot(self):
    if self.current_img is None: return
    gray = cv2.cvtColor(self.current_img, cv2.COLOR_BGR2GRAY)
    thymio_mask = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)
    opening = cv2.morphologyEx(thymio_mask, cv2.MORPH_OPEN, np.ones((5,5), np.uint8), iterations=7) 
    contours= cv2.findContours(opening,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[0]
    M = cv2.moments(contours[0]) 
    cx = int(M["m10"] / M["m00"])
    cy = int(M["m01"] / M["m00"])
    h, w = gray.shape[:2]
    self.img_ratio = w/h
    hb, wb = (int(self.map_size/self.img_ratio),self.map_size)
    self.robot_pos = (int(cx/h*hb),int(cy/w*wb))
    self.robot_angle = self.rectifyAngle(cv2.minAreaRect(contours[0]))
```

### Obstacle detection and map generation
The same method as for the detection of the Thymio is applied to detect the obstacles but with an inversed binary thresholding instead. The image is also post processed for denoising and filling the possible gaps in the obstacles. The mask is then turned into a binary map and rescaled to the desired size. The same function as before were used plus the following function to resize the binary map:
- Resizing the map: `skimage.transform.resize`

Below is an exemple of the obstacles detection and map creation
<center><img src="images\vision_obstacle_original.png"></center>
<center>Original image from the camera</center>

<center><img src="images\vision_obstacle_mask_noisy.png"></center>
<center>Threshold to obtain the obstacles</center>

<center><img src="images\vision_obstacle_mask_fill.png"></center>
<center>Process to fill the gaps</center>

<center><img src="images\vision_obstacle_mask_denoise.png"></center>
<center>Denoise the mask</center>

<center><img src="images\vision_obstacle_mask_binary.png"></center>
<center>Turn into a binary map</center>

<center><img src="images\vision_obstacle_mask_rescale.png"></center>
<center>Resize to de desired map size</center>

The function used to detect the obstacles and create the map for the global navigation is implemented as follows: 
```python
def createMap(self):
    if self.current_img is None: return
    gray = cv2.cvtColor(self.current_img,cv2.COLOR_BGR2GRAY)
    obst_mask = cv2.threshold(gray, 20, 255, cv2.THRESH_BINARY_INV)[1]
    closing = cv2.morphologyEx(obst_mask, cv2.MORPH_CLOSE, np.ones((5,5), np.uint8), iterations=15) 
    opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN,  np.ones((5,5), np.uint8), iterations=7) 
    h, w = gray.shape[:2]
    self.img_ratio = w/h
    binary_map = img_as_bool(resize(opening/opening.max(),(int(self.map_size/self.img_ratio),self.map_size)))
    self.map = binary_map.astype(int)
```


## Filtering

## Global Navigation

## Local Navigation

The Local Navigation module is responsible for:
- Finding the right speed value of the motors to reach the goal
- Chose whether to follow the path created from the A* algorithm or to avoid a local obstacle

A local obstacle is a 3D obstacle which wasn't present on the map at the moment of calculating the planning for the optimal path. We chose to have it white for a better detection from the sensors.

| **Useful constants**                            | **Purpose**                                                                |
|---------------------------|-------------------------------------------------------------------------------------------------|
| obstThrL = 500    | This constant is used to know if there are no local objects in front of the Thymio. The value should be 0, but we have a higher value to avoid disturbances.    |
| obstThrH = 900    | This constant is used to detect an obstacle near the proximity sensors. The range is [1000, 4500] when he is detecting something and a high value means that the obstacle is nearer to the Thymio. The range is [1, 14] cm   |
| obstSpeedGain = [0.1, 0.06, 0, -0.06, -0.1]     | This is a vector containing the necessary gains for motor speed to avoid obstacles while navigating with local navigation. They are related to the first 5 proximity sensors from front left to front right.|

### General Local Navigation functions
The Local Navigation module was made without a class. The main functions provided by the Local Navigation are as follow:


| **Function**           | **Role**                                                                  |
|------------------------|---------------------------------------------------------------------------|
| obstacleAvoidance()    | It has as parameters the position of the Thymio (position_x, position_y) and returns saved_pos_idx which corrisponds to the index of the coordinates from path matrix. This position is the one of the Thymio when he first detect a local obstacle         |
| checkState()           | Returns the current position of the robot in the map (in term of cells)   |
| currentPositionIndex() | Returns the index from path matrix of the current position                |
| motionControl()        | Returns motors left and right speed                                       |

### Local obstacle detection 

The algorithm to detect the local obstacle works in this way: 
- Proximity sensors refresh their values continuously while following the path generated from A star algorithm. 
- Once a local obstacle is detected, the proximity sensor value pass a threshold value of 900, which switch the mode from A star algorithm to obstacle avoidance.

### Local obstacle circumnavigation

In order to pass an obstacle the Thymio save his position when the obstacle is detected. After this he verifies if the obstacle is on his right or his left by compare values of proximity sensors: a greater value means a closer obstacle and the Thymio will first turn on the opposite direction. For instance if the value on the right are bigger than the one on the left, then he will take a clockwise direction by increasing the speed of the right motor and decrease the speed of the left motor.



<center><img src="images\obstacle_detection.png" width = 400></center>
<center> The moment of obstacle detection at a distance of approximately 14 cm </center>

This operation is done until the sensors are free again as shown in the image below.

<center><img src="images\obstacle_avoided.png" width = 400></center>
<center> No more obstacle detected </center>

From this moment the robot will continue turning, at a fixed speed of 200 for the external motor and 100 for the internal motor, and looking for one of the coordinates from the original path planning. If his positon is again on one of the coordinates from path planning, then he switch his mode from obstacle avoidance to A star algorithm.  

This algorithm is responsible for finding the original path.
Here is very important to have the saved position and his corresponding index, when the local obstacle is detected (saved_pos and saved_pos_idx). 
The resoning is that from the saved index we check all the cosecutive coordinates until the final objective, except from ###, to compare the current position of the Thymio and the position of all the possibile targets. If they match it means that the Thymio circumnavigated the obstacle and he can continue on the original path. 
```python
def funtion()
```

## Conclusion
we'll discuss the conclusion

## Run the project

*insert code here*